# Test standard API

In [13]:
from openai import OpenAI

# Set your OpenAI API key
OPENAI_API_KEY = ''

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  temperature=0.0,
  messages=[
	{"role": "system", "content": "Du er en profesjonell journalist som skriver anmeldelser av filmer og TV-serier."},
    {"role": "user", "content": "Skriv en anmeldelse av filmen «The Expendables 2» på rundt 300 ord hvor du gir terningkast 4"},

  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='"The Expendables 2" er en actionfylt film som samler en imponerende rollebesetning av actionstjerner fra 80- og 90-tallet. Med navn som Sylvester Stallone, Arnold Schwarzenegger, Bruce Willis, Jean-Claude Van Damme og Chuck Norris, er det ingen tvil om at denne filmen er fullpakket med action og eksplosjoner.\n\nHandlingen følger Stallones karakter, Barney Ross, og hans team av leiesoldater som blir satt på oppdrag for å hente tilbake en verdifull gjenstand fra en skurkaktig russisk general spilt av Van Damme. Filmen byr på en rekke spektakulære actionscener, inkludert intense skuddvekslinger, biljakter og eksplosjoner som vil glede fans av sjangeren.\n\nSelv om filmen leverer på actionfronten, mangler den dessverre substans når det gjelder historie og karakterutvikling. Dialogen er klisjéfylt og overfladisk, og karakterene blir ikke gitt mye dybde eller motivasjon. Dette gjør det vanskelig å virkelig bry seg om hva som skjer med dem gjennom filmen.\n\nSk

# Using the batch API

### Make JSON file with prompts

In [42]:
import json
import pandas as pd

prompts_df = pd.read_csv('prompt_set_4.csv')

# System prompt that will precede each user prompt
system_prompt = {"role": "system", "content": "Du er en profesjonell journalist som skriver anmeldelser av filmer og TV-serier."}

# Prepare the .jsonl content
jsonl_content = ""
for index, row in prompts_df.iterrows():
    request = {
        "custom_id": f"request-{row['id']}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4-turbo",
            "temperature": 0.5,
            "messages": [system_prompt, {"role": "user", "content": row['prompt']}]
        }
    }
    jsonl_content += json.dumps(request) + "\n"

# Save to a .jsonl file
jsonl_file_path = 'prompt_set_4_temp_0point5.jsonl'
with open(jsonl_file_path, 'w') as f:
    f.write(jsonl_content)


### Upload JSON file

In [43]:
from openai import OpenAI

OPENAI_API_KEY = ''
client = OpenAI(api_key=OPENAI_API_KEY)

batch_input_file = client.files.create(
  file=open("prompt_set_4_temp_0point5.jsonl", "rb"),
  purpose="batch"
)

### Create batch

In [44]:
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "prompt set 4 and temperature 0.5 with 4-turbo model"
    }
)

Batch(id='batch_TrrtZfrzt2brcK1hpzfTFi6W', completion_window='24h', created_at=1715265930, endpoint='/v1/chat/completions', input_file_id='file-GTz6a8pGMFIMgOoVKrxUvJJX', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1715352330, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'prompt set 4 and temperature 0.5 with 4-turbo model'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

### Check status of batch

In [45]:
client.batches.retrieve("batch_TrrtZfrzt2brcK1hpzfTFi6W")

Batch(id='batch_TrrtZfrzt2brcK1hpzfTFi6W', completion_window='24h', created_at=1715265930, endpoint='/v1/chat/completions', input_file_id='file-GTz6a8pGMFIMgOoVKrxUvJJX', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='duplicate_custom_id', line=853, message='The custom_id for this request is a duplicate of another request. The custom_id parameter must be unique for each request in a batch.', param='custom_id')], object='list'), expired_at=None, expires_at=1715352330, failed_at=1715265930, finalizing_at=None, in_progress_at=None, metadata={'description': 'prompt set 4 and temperature 0.5 with 4-turbo model'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

### Retrieve results

In [ ]:
import json

file_response = client.files.content("batch_TrrtZfrzt2brcK1hpzfTFi6W")
data = file_response.text

json_data = json.loads(data)

with open('prompt_set_4_temp_0point5_responses.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=4)